In [1]:
from utils import *
import pandas as pd
import uuid
run_hash = uuid.uuid4().hex
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
lang = "HR.HT"
config = democonfig
config["NUM_EPOCH"] = 30
config["lang"] = lang
config["run"] = run_hash

config["model_name"] = "classla/bcms-bertic"
config["model_type"] = "bert"
test = (
    pd.read_excel("COPA-MT-MK.BG.TR.MT.IS.HR.xlsx", sheet_name="test")
    .dropna(axis=1, how="all")
    .rename(columns=col_rename_dict)
)
dev = (
    pd.read_excel("COPA-MT-MK.BG.TR.MT.IS.HR.xlsx", sheet_name="dev")
    .dropna(axis=1, how="all")
    .rename(columns=col_rename_dict)
)
train = (
    pd.read_excel("COPA-MT-MK.BG.TR.MT.IS.HR.xlsx", sheet_name="train")
    .dropna(axis=1, how="all")
    .rename(columns=col_rename_dict)
)

for asksfor in reversed("cause effect".split()):
    import torch
    torch.cuda.empty_cache()
    import gc
    gc.collect()
    model = instantiate_model(**config)
    sep_token = model.tokenizer.sep_token
    config["asksfor"] = asksfor
    train_subset = filter_dataframe(
        train,
        lang=lang,
        asksfor=asksfor,
        sep_token= sep_token,
        )
    test_subset = filter_dataframe(
        test,
        lang=lang,
        asksfor=asksfor,
        sep_token=model.tokenizer.sep_token,
        )
    dev_subset = filter_dataframe(
        dev,
        lang=lang,
        asksfor=asksfor,
        sep_token=model.tokenizer.sep_token,
        )

    model = train_model_on(model, train_subset, NUM_EPOCH=config.get("NUM_EPOCH"))
    y_pred_test = eval_model(model, test_subset)
    y_true_test = test.loc[test["asks-for"] == asksfor, "gold"].values.tolist()
    from sklearn.metrics import accuracy_score
    print(f"On test: {accuracy_score(y_true_test, y_pred_test)}")
    results = dict(config)
    results["accuracy"] = accuracy_score(y_true_test, y_pred_test)
    results["split"] = "test"
    results["y_true"] = y_true_test
    results["y_pred"] = y_pred_test
    with open("002_results_new_script.jsonl", "a") as f:
        f.write(str(results)+'\n')

    y_pred_dev = eval_model(model, dev_subset)
    y_true_dev = dev.loc[dev["asks-for"] == asksfor, "gold"].values.tolist()
    print(f"On dev: {accuracy_score(y_true_dev, y_pred_dev)}")
    results = dict(config)
    results["accuracy"] = accuracy_score(y_true_dev, y_pred_dev)
    results["split"] = "dev"
    results["y_true"] = y_true_dev
    results["y_pred"] = y_pred_dev
    with open("002_results_new_script.jsonl", "a") as f:
        f.write(str(results)+'\n')
    del model

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing BertForSequenceClassification: ['electra.encoder.layer.3.attention.self.query.bias', 'electra.encoder.layer.11.intermediate.dense.weight', 'electra.encoder.layer.11.attention.output.LayerNorm.bias', 'electra.encoder.layer.1.attention.self.value.bias', 'electra.encoder.layer.11.attention.output.LayerNorm.weight', 'electra.encoder.layer.6.output.dense.bias', 'electra.encoder.layer.2.output.LayerNorm.bias', 'electra.encoder.layer.8.attention.self.value.weight', 'electra.encoder.layer.4.attention.output.dense.weight', 'electra.encoder.layer.0.intermediate.dense.bias', 'electra.encoder.layer.7.attention.output.dense.weight', 'electra.embeddings.position_ids', 'electra.encoder.layer.2.attention.self.key.bias', 'electra.encoder.layer.3.intermediate.dens

  0%|          | 0/250 [00:00<?, ?it/s]

On test: 0.464


  0%|          | 0/48 [00:00<?, ?it/s]

On dev: 0.5


You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing BertForSequenceClassification: ['electra.encoder.layer.3.attention.self.query.bias', 'electra.encoder.layer.11.intermediate.dense.weight', 'electra.encoder.layer.11.attention.output.LayerNorm.bias', 'electra.encoder.layer.1.attention.self.value.bias', 'electra.encoder.layer.11.attention.output.LayerNorm.weight', 'electra.encoder.layer.6.output.dense.bias', 'electra.encoder.layer.2.output.LayerNorm.bias', 'electra.encoder.layer.8.attention.self.value.weight', 'electra.encoder.layer.4.attention.output.dense.weight', 'electra.encoder.layer.0.intermediate.dense.bias', 'electra.encoder.layer.7.attention.output.dense.weight', 'electra.embeddings.position_ids', 'electra.encoder.layer.2.attention.self.key.bias', 'electra.encoder.layer.3.intermediate.dens

In [ ]:
import pandas as pd

from ast import literal_eval
with open("002_results_new_script.jsonl") as f:
    content = f.readlines()
    content = map(literal_eval, content)
pd.DataFrame(data=content)

In [ ]:
asksfor

In [ ]:
dev[dev["asks-for"] == asksfor].shape